# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [2]:
df_full = pd.read_csv('autos.csv')  # jupyterhub постоянно вылетал при подборе гиперпараметров, 
                                    # поэтому я вынужден был перейти в локальный jupyter и изменить путь к данным.

In [3]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df_full.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


### Обработка пропусков и выбросов

На основании полученной информации о датасете, можем запланировать следующие действия:
   - Столбцы `DateCrawled`, `NumberOfPictures`, `PostalCode`, `LastSeen`, `DateCreated` удалим, как содержащие информацию не влияющую на предсказание цены.
   - В данных есть пропуски, удалим их или заполним.
   - В столбцах, содержащих числовые данные, есть выбросы, отбросим их как ошибочные данные.
   - К столбцам типа object применим прямое кодирование, после этого ко всем данным (кроме цели) применим масштабирование.   

In [5]:
print('Количество строк содержащих цену равную 0 -', df_full[df_full['Price'] == 0].shape[0])

Количество строк содержащих цену равную 0 - 10772


Нам необходимо предсказать рыночную цену автомобиля. Цену равную 0, будем считать ошибкой ввода данных, заполнить можно медианной ценой по таким же автомобилям. Но признаков для определения такого же автомобиля слишком много, поэтому самих автомобилей в подборке будет мало, поэтому полученная медианная цена может быть не объективна. С другой стороны данных с нулевой ценой 3%. Можем удалить эти данные, что бы они не попали в тренировочную выборку и своей нулевой ценой не влияли на результат. После обучения модели, можно попробовать предсказать цену на эти машины.

In [6]:
df = df_full[df_full['Price'] != 0]

In [7]:
df.query('RegistrationYear < 1900 or RegistrationYear > 2016')['RegistrationYear'].count()

13832

Количество машин с некорректным годом регистрации - 13832. Год регистрации (выпуска) - существенный признак машины, удалим эти данные.

In [8]:
df = df.query('1900 < RegistrationYear < 2017')
df.query('RegistrationYear < 1900 or RegistrationYear > 2016')['RegistrationYear'].count()

0

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329765 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        329765 non-null  object
 1   Price              329765 non-null  int64 
 2   VehicleType        309828 non-null  object
 3   RegistrationYear   329765 non-null  int64 
 4   Gearbox            314163 non-null  object
 5   Power              329765 non-null  int64 
 6   Model              314128 non-null  object
 7   Kilometer          329765 non-null  int64 
 8   RegistrationMonth  329765 non-null  int64 
 9   FuelType           305590 non-null  object
 10  Brand              329765 non-null  object
 11  Repaired           269883 non-null  object
 12  DateCreated        329765 non-null  object
 13  NumberOfPictures   329765 non-null  int64 
 14  PostalCode         329765 non-null  int64 
 15  LastSeen           329765 non-null  object
dtypes: int64(7), object(

Видим, что есть пропуски в категорийных данных в столбцах `VehicleType`, `Gearbox`, `Model`, `FuelType`, `Repaired`.  
Заполним пропуски значением `unknown`.

In [10]:
df = df.fillna('unknown')

Проверим сколько машин с некорректно указанной мощностью двигателя (меньше 30 л.с и более 1000 л.с.). 

In [11]:
df.query('Power < 30 or Power > 1000')['Power'].count()

33963

Не корректные данные указаны в 33963 записях. Данных 11%, их удаление может повлиять на результат, заменим не корректные значения `Power` медианным значением `Power` сгруппированным по `VehicleType`.

In [12]:
power_median = df.query('Power > 30 and Power < 1000').groupby('VehicleType')['Power'].median()
df.loc[(df['Power'] < 30) | (df['Power'] > 1000) , ['Power']] = \
                    power_median[df.loc[(df['Power'] < 30) | (df['Power'] > 1000),'VehicleType']].values

Удалим столбцы `DateCrawled`, `NumberOfPictures`, `PostalCode`, `LastSeen`, `DateCreated`, как не влияющие на анализ.

In [13]:
df = df.drop(['DateCrawled', 'NumberOfPictures', 'PostalCode', 'LastSeen', 'DateCreated'], axis=1)

In [14]:
#df['RegistrationMonth'].value_counts(dropna = False)

В столбце месяц регистрации есть нулевые данные. Это некорректно. Но данных достаточно много (32772) для удаления. По идее нужно заполнить, но чем заполнять не понятно. Среднее и медиана не подходят, так как просто увеличат на 32 тыс значение за какой-нибудь месяц. Можно в случайном порядке распределить по остальным месяцам, но такой случайный выбор никакой пользы для предсказаний не принесет. Оставим как есть, будем считать что 0 месяц означает отсутствие месяца.

In [15]:
df = df.drop(['RegistrationMonth'], axis=1)

### Кодирование и масштабирование

Данные подготовили, теперь закодируем категорийные признаки и масштабируем данные.

In [16]:
num = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth']
kat = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

In [17]:
feature = df.drop(['Price'], axis=1)
target = df['Price']

In [18]:
feature_train, feature_test, target_train, target_test = train_test_split(feature, target, test_size=0.2, random_state=68756)
feature_train, feature_valid, target_train, target_valid = \
                                            train_test_split(feature_train, target_train, test_size=0.25, random_state=68756)

Кодируем категорийные признаки.

In [19]:
OE = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype='int64')

In [20]:
feature_train[kat] = OE.fit_transform(feature_train[kat])
feature_valid[kat] = OE.transform(feature_valid[kat])
feature_test[kat] = OE.transform(feature_test[kat])

Масштабируем все признаки.

In [21]:
scaler = StandardScaler()
feature_train = scaler.fit_transform(feature_train)
feature_valid = scaler.transform(feature_valid)
feature_test = scaler.transform(feature_test)

**Вывод:**  
Данные разбиты на выборки и подготовлены для обучения моделей.  
Пропуски удалены или заполнены, незначащие столбцы удалены.  
Данные закодированы и масштабированы.

## Обучение моделей

Будем обучать 4 модели:
- Линейную регрессию
- Случайный лес
- LightGBM
- CatBoost

### Линейная регрессия

In [22]:
model_lr = LinearRegression(n_jobs=-1)

In [23]:
%%time

model_lr.fit(feature_train, target_train)
pass

Wall time: 45.8 ms


In [24]:
%%time

predict_lr = model_lr.predict(feature_valid)

Wall time: 998 µs


In [25]:
print('RMSE =', int((mean_squared_error(target_valid, predict_lr))**0.5))

RMSE = 2986


Линейная регресси с параметрами по умолчанию показала следующие результаты:  
Время обучения - 30 мс  
Время предсказания - 7 мс  
RMSE - 3036

```python
# прямое кодирование (для линейных моделей)
features_train_ohe = pd.get_dummies(features_train, drop_first=True)
features_valid_ohe = pd.get_dummies(features_valid, drop_first=True)
features_test_ohe = pd.get_dummies(features_test, drop_first=True)
# после прямого кодирования кол-во признаков в выборках будет отличаться
# поэтому стоит удалить лишние признаки в каждой выборке

# Найдем общие признаки между обучающей, валидационной и тестовой выборками
common_columns = set(features_train_ohe.columns) & set(features_valid_ohe.columns) & set(features_test_ohe.columns)

# Оставим в каждой выборке только общие признаки
features_train_ohe = features_train_ohe[list(common_columns)]
features_valid_ohe = features_valid_ohe[list(common_columns)]
features_test_ohe = features_test_ohe[list(common_columns)]
```

In [26]:
feature_train_d, feature_test_d, target_train_d, target_test_d = train_test_split(feature, target, test_size=0.2, random_state=68756)
feature_train_d, feature_valid_d, target_train_d, target_valid_d = \
                                            train_test_split(feature_train_d, target_train_d, test_size=0.25, random_state=68756)

In [27]:
features_train_ohe = pd.get_dummies(feature_train_d, drop_first=True)
features_valid_ohe = pd.get_dummies(feature_valid_d, drop_first=True)
features_test_ohe = pd.get_dummies(feature_test_d, drop_first=True)
common_columns = set(features_train_ohe.columns) & set(features_valid_ohe.columns) & set(features_test_ohe.columns)
features_train_ohe = features_train_ohe[list(common_columns)]
features_valid_ohe = features_valid_ohe[list(common_columns)]
features_test_ohe = features_test_ohe[list(common_columns)]

In [28]:
lr = LinearRegression(n_jobs=-1).fit(features_train_ohe, target_train)

In [29]:
print('RMSE =', int((mean_squared_error(target_valid, lr.predict(features_valid_ohe)))**0.5))

RMSE = 2673


#### Подберем гиперпараметры для Линейной регрессии

In [30]:
%%time

rmse = 100000
best_model_lr = 0

for i in range(0, 1000, 10):
    model_lr = LinearRegression(n_jobs=i)
    model_lr.fit(feature_train, target_train)
    predict_lr = model_lr.predict(feature_valid)
    rmse_loc = int((mean_squared_error(target_valid, predict_lr))**0.5)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_lr = model_lr
print('Лучшая модель:', best_model_lr)
print('Лучшая RMSE:', rmse)

Лучшая модель: LinearRegression(n_jobs=0)
Лучшая RMSE: 2986
Wall time: 2.78 s


Результат линейной регресси с подбираемыми параметрами не улучшился.  
Среднее время обучения и предсказания - 20 мс  
RMSE - 3036

Проверим результат лучшей модели линейной регрессии

In [31]:
%%time

best_model_lr.fit(feature_train, target_train)
pass

Wall time: 28.9 ms


In [32]:
%%time

best_model_lr = model_lr.predict(feature_valid)

Wall time: 2 ms


In [33]:
print('RMSE =', int((mean_squared_error(target_valid, predict_lr))**0.5))

RMSE = 2986


Лучшая модель линейной регресси показала следующие результаты:  
Время обучения - 28.5 мс  
Время предсказания - 5.85 мс  
RMSE - 3036

### Случайный лес

In [34]:
model_rf = RandomForestRegressor(random_state=68756)

In [35]:
%%time

model_rf.fit(feature_train, target_train)
pass

Wall time: 44.8 s


In [36]:
%%time

predict_rf = model_rf.predict(feature_valid)

Wall time: 2.18 s


In [37]:
print('RMSE =', int((mean_squared_error(target_valid, predict_rf))**0.5))

RMSE = 1641


Случайный лес с параметрами по умолчанию показал следующие результаты:  
Время обучения - 1 минута  
Время предсказания - 3 с  
RMSE - 1693

#### Подберем гиперпараметры для Случайного леса

Так как модель при большом значении параметров считается медленно, параметры подберем по отдельности.

In [38]:
def giper(model):
    model.fit(feature_train, target_train)
    predict = model.predict(feature_valid)
    return int((mean_squared_error(target_valid, predict))**0.5)

def print_best(best_model, rmse):
    print('Лучшая модель:', best_model)
    print('Лучшая RMSE:', rmse)

In [39]:
rmse = 100000
best_model_rf = 0

for feat in tqdm(range(1, 8)):
    model = RandomForestRegressor(n_estimators=20, max_features=feat, random_state=68756)
    rmse_loc = giper(model)
#    print(rmse_loc)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_rf = model

print_best(best_model_rf, rmse)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:37<00:00,  5.33s/it]

Лучшая модель: RandomForestRegressor(max_features=3, n_estimators=20, random_state=68756)
Лучшая RMSE: 1638


Лучшее значение RMSE = 1691, при max_features = 4

In [40]:
rmse = 100000
best_model_rf = 0

for depth in tqdm(range(2, 30)):
    model = RandomForestRegressor(n_estimators=20, max_depth=depth, max_features=4, random_state=68756)
    rmse_loc = giper(model)
#    print(rmse_loc)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_rf = model

print_best(best_model_rf, rmse)

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [01:33<00:00,  3.35s/it]

Лучшая модель: RandomForestRegressor(max_depth=19, max_features=4, n_estimators=20,
                      random_state=68756)
Лучшая RMSE: 1618


Лучшее значение RMSE = 1671, при max_depth = 21 и max_features = 4

In [41]:
rmse = 100000
best_model_rf = 0

for n in tqdm(range(100, 301, 100)):
    model = RandomForestRegressor(n_estimators=n, max_depth=21, max_features=4, random_state=68756)
    rmse_loc = giper(model)
#    print(rmse_loc)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_rf = model

print_best(best_model_rf, rmse)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:20<00:00, 46.95s/it]

Лучшая модель: RandomForestRegressor(max_depth=21, max_features=4, n_estimators=300,
                      random_state=68756)
Лучшая RMSE: 1592


Лучшее значение RMSE = 1643, при n_estimators=200, max_depth = 21 и max_features = 4              

Проверим время работы модели Случайный лес с подобранными гиперпараметрами.

In [42]:
%%time

best_model_rf.fit(feature_train, target_train)
pass

Wall time: 1min 3s


In [43]:
%%time

predict_rf = best_model_rf.predict(feature_valid)

Wall time: 5.04 s


In [44]:
print('RMSE =', int((mean_squared_error(target_valid, predict_rf))**0.5))

RMSE = 1592


Случайный лес с подобранными гиперпараметрами показал следующие результаты:  
Время обучения - 48 c  
Время предсказания - 4.15 с  
RMSE - 1643

### LightGBM

In [45]:
model_lgbm = LGBMRegressor(random_state=68756)

In [46]:
%%time

model_lgbm.fit(feature_train, target_train)
pass

Wall time: 458 ms


In [47]:
%%time

predict_lgbm = model_lgbm.predict(feature_valid)

Wall time: 121 ms


In [48]:
print('RMSE =', int((mean_squared_error(target_valid, predict_lgbm))**0.5))

RMSE = 1712


LightGBM с параметрами по умолчанию показал следующие результаты:  
Время обучения - 1.5 минуты  
Время предсказания - 0.5 с  
RMSE - 1790

#### Подберем гиперпараметры для LightGBM

In [49]:
rmse = 100000
best_model_lgbm = 0

for n in tqdm(range(10, 13)):
    model = LGBMRegressor(max_depth=n, random_state=68756)
    rmse_loc = giper(model)
    print(rmse_loc)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_lgbm = model
        print(n, '-', rmse)

print_best(best_model_lgbm, rmse)

 33%|████████████████████████████                                                        | 1/3 [00:00<00:01,  1.78it/s]

1711
10 - 1711


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:01<00:00,  1.71it/s]

1707
11 - 1707


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.65it/s]

1708
Лучшая модель: LGBMRegressor(max_depth=11, random_state=68756)
Лучшая RMSE: 1707


Лучшее значение RMSE = 1786, при max_depth=11

In [50]:
rmse = 100000
best_model_lgbm = 0

for n in tqdm(range(100, 3101, 1000)):
    model = LGBMRegressor(n_estimators=n, max_depth=11, random_state=68756)
    rmse_loc = giper(model)
    print(rmse_loc)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_lgbm = model
        print(n, '-', rmse)

print_best(best_model_lgbm, rmse)

 25%|█████████████████████                                                               | 1/4 [00:00<00:01,  1.61it/s]

1707
100 - 1707


 50%|██████████████████████████████████████████                                          | 2/4 [00:04<00:04,  2.41s/it]

1567
1100 - 1567


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:11<00:04,  4.51s/it]

1553
2100 - 1553


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:22<00:00,  5.51s/it]

1551
3100 - 1551
Лучшая модель: LGBMRegressor(max_depth=11, n_estimators=3100, random_state=68756)
Лучшая RMSE: 1551


Лучшее значение RMSE = 1600, при n_estimators = 3100 и max_depth=11

Проверим время работы модели LightGBM с подобранными гиперпараметрами.

In [51]:
final_model_lgbm = LGBMRegressor(n_estimators=3100, max_depth=11, random_state=68756)

In [52]:
%%time

final_model_lgbm.fit(feature_train, target_train)
pass

Wall time: 7.75 s


In [53]:
%%time

predict_lgbm = final_model_lgbm.predict(feature_valid)

Wall time: 3.08 s


In [54]:
print('RMSE =', int((mean_squared_error(target_valid, predict_lgbm))**0.5))

RMSE = 1551


LightGBM с подобранными гиперпараметрами показал следующие результаты:  
Время обучения - 7 c  
Время предсказания - 3 с  
RMSE - 1600   

### CatBoost

In [55]:
model_cat = CatBoostRegressor(random_state=68756)

In [56]:
%%time

model_cat.fit(feature_train, target_train, verbose=False, plot=True)
pass

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Wall time: 13.7 s


In [57]:
%%time

predict_cat = model_cat.predict(feature_valid)

Wall time: 31.9 ms


In [58]:
print('RMSE =', int((mean_squared_error(target_valid, predict_cat))**0.5))

RMSE = 1619


CatBoost с параметрами по умолчанию показал следующие результаты:   
Время обучения - 27 c  
Время предсказания - 62 мс  
RMSE - 1695  

#### Подберем гиперпараметры для CatBoost

In [59]:
rmse = 100000
best_model_cat = 0

for n in tqdm(range(12, 15)):
    model = CatBoostRegressor(max_depth=n, verbose=False, random_state=68756)
    rmse_loc = giper(model)
#    print(rmse_loc)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_cat = model
        print(n, '-', rmse)

print_best(best_model_cat, rmse)

 33%|████████████████████████████                                                        | 1/3 [01:00<02:01, 60.79s/it]

12 - 1558


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [05:41<00:00, 113.94s/it]

Лучшая модель: <catboost.core.CatBoostRegressor object at 0x000001890001F700>
Лучшая RMSE: 1558


Лучшее значение RMSE = 1615, при max_depth = 13

In [60]:
rmse = 100000
best_model_cat = 0

for n in tqdm(range(1100, 3101, 1000)):
    model = CatBoostRegressor(n_estimators=n, max_depth=13, verbose=False, random_state=68756)
    rmse_loc = giper(model)
#    print(rmse_loc)
    if rmse_loc < rmse:
        rmse = rmse_loc
        best_model_cat = model
        print(n, '-', rmse)

print_best(best_model_cat, rmse)

 33%|███████████████████████████▋                                                       | 1/3 [01:48<03:36, 108.30s/it]

1100 - 1558


 67%|███████████████████████████████████████████████████████▎                           | 2/3 [05:11<02:44, 164.35s/it]

2100 - 1553


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [10:05<00:00, 201.73s/it]

Лучшая модель: <catboost.core.CatBoostRegressor object at 0x000001890001FA90>
Лучшая RMSE: 1553


Лучшее значение RMSE = 1605, при n_estimators = 3100

Проверим время работы модели CatBoost с подобранными гиперпараметрами.

In [61]:
best_model_cat = CatBoostRegressor(n_estimators=3100, max_depth=13, verbose=False, random_state=68756)

In [62]:
%%time

best_model_cat.fit(feature_train, target_train)
pass

Wall time: 4min 56s


In [63]:
%%time

predict_cat = best_model_cat.predict(feature_valid)

Wall time: 825 ms


In [64]:
print('RMSE =', int((mean_squared_error(target_valid, predict_cat))**0.5))

RMSE = 1553


CatBoost с подобранными гиперпараметрами показал следующие результаты:  
Время обучения - 6 минут  
Время предсказания - 2 с  
RMSE - 1603  

**Вывод:**  
Обучили 4 модели.  
Самая быстрая - линейная регрессия.  
Самая точная - LightGBM.

## Анализ моделей

В результате тестирования моделей и подбора гиперпараметров наблюдаем следующие результаты:

Лучшая модель линейной регресси - LinearRegression(n_jobs=0)  
Время обучения - 28.5 мс  
Время предсказания - 5.85 мс  
RMSE - 3036

Лучшая модель случайный лес - RandomForestRegressor(max_depth=21, max_features=4, n_estimators=200, random_state=68756)  
Время обучения - 48 c  
Время предсказания - 4.15 с  
RMSE - 1643  

Лучшая модель LightGBM - LGBMRegressor(max_depth=11, n_estimators=3100, random_state=68756)  
Время обучения - 7 c  
Время предсказания - 3 с  
RMSE - 1600

Лучшая модель CatBoost - CatBoostRegressor(max_depth=13, n_estimators=3000, random_state=68756)  
Время обучения - 6 минут  
Время предсказания - 2 с  
RMSE - 1605  

|Модель|Время обучения|Время предсказания|RMSE|  
|:-|:-:|:-:|:-:|  
|Линейная регрессия|29 мс|6 мс|3036|     
|Случайный лес|48 с|4 с|1643|    
|LightGBM|7 с|3 с|1600|     
|CatBoost|6 мин|2 с|1605|    





Лучший результат показывает модель LightGBM. CatBoost показывает почти такой же результат, но обучается медленнее, предсказывает быстрее. Возможно я подбирал не те гиперпараметры.  

От модели CatBoost, можно сказать следующее, если не улучшать значения гиперпараметров с целью получить несколько дополнительных единиц RMSE (бесполезных в практическом применении), то можно значительно сократить время обучения модели.  
Например:  
при  n_estimators = 1000 (значение по умолчанию), время обучения 2 минуты, RMSE - 1615  
при  n_estimators = 400, время обучения 46 с, RMSE - 1628  
Но при этом результат оказывается, ниже, чем у LightGBM.

Самый быстрый результат дает модель линейная регрессия, но качество предсказаний не соответствует условию задания.

## Тестирование лучшей модели

Для финальной проверки на тестовой выборке возьмем модель LightGBM.  
LGBMRegressor(max_depth=11, n_estimators=3100)

In [65]:
%%time

final_model_lgbm.fit(feature_train, target_train)
pass

Wall time: 6.53 s


In [66]:
%%time

predict_lgbm_test = final_model_lgbm.predict(feature_test)

Wall time: 2.96 s


In [67]:
print('RMSE =', int((mean_squared_error(target_test, predict_lgbm_test))**0.5))

RMSE = 1562


**Вывод:**  
Лучшая модель LightGBM - LGBMRegressor(max_depth=11, n_estimators=3100)  
Лучший результат RMSE - 1573

**Вывод:**  
В результате проекта подготовили данные, обучили 4 модели и выбрали лучшую под требования заказчика.

**Вывод:**  
В процессе работы были выполнены следующие этапы:
- Обработка данных
    - Получение данных
    - Удаление некорректных данных
    - Заполнение пропусков
    - Замена, где возможно, некорректых данных на значения адекватные по смыслу
    - Удалили столбцы не влияющие на предсказания
    - Кодирование категорийных признаков
    - Масштабирование всех признаков
- Обучение моделей
    - Обучили 4 модели
- Анализ моделей
    - Выбрали лучшую модель и протестировали ее